In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine, text
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"
from datim import datim_query

In [2]:
_QUERY="""
select * from patient;
"""
QUERY="""
SELECT
    p.patient_code,
    tmi.*
FROM
    tracking_motherbasicinfo tmi
LEFT JOIN
    patient p 
ON
    p.id = tmi.id_patient;
"""

MUSO_WITH_COMPTAGEMENAGE = """
SELECT *
FROM
    caris_db.view_muso_household_report
where (!isnull(indice_householdcount) or (is_household_applicable ='yes' ))
"""

""" INFO="mother_tracking_info"
_INFO="patient"
dataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) """

' INFO="mother_tracking_info"\n_INFO="patient"\ndataset.to_excel(f"../../pvvih_sansComptage/{_INFO}.xlsx",index=False) '

In [3]:
# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

# Non-applicable 
MHC22 = f"""
SELECT 
    b.caseid, mhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (!ISNULL(indice_householdcount)
            OR (is_household_applicable = 'yes'))) mhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mhc22.patient_code = b.patient_code
"""



# applicable 
NOMHC22 = f"""
SELECT 
    b.caseid, nomhc22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.view_muso_household_report
    WHERE
        (ISNULL(indice_householdcount)
            AND (is_household_applicable = 'no'
            OR ISNULL(is_household_applicable)))) nomhc22
        LEFT JOIN
    (SELECT 
        patient_code, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON nomhc22.patient_code = b.patient_code
"""

MUSO_HOUSEHOLD_22 = f"""
SELECT 
    b.caseid, mh22.*
FROM
    (SELECT 
        *
    FROM
        caris_db.muso_household_2022) mh22
        LEFT JOIN
    (SELECT 
        id, muso_case_id AS caseid
    FROM
        caris_db.patient) b ON mh22.id_patient = b.id
"""

In [4]:
HUBS = f"""
SELECT 
    dh.name AS hub,
    dh.address,
    lc.name AS commune,
    ld.name AS departement,
    dh.is_inactive,
    dh.inactive_date,
    dh.creation_date,
    dh.created_at
FROM
    caris_db.dream_hub dh
        LEFT JOIN
    caris_db.lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    caris_db.lookup_departement ld ON ld.id = lc.departement
GROUP BY hub
HAVING hub != ''
"""

In [5]:
from enum import Enum

In [6]:
class EidDate(Enum):
    start_date="2023-10-01"
    end_date="2024-04-30"

'USE caris_db;'

EID = f"""
select Patient_code,Code_mere,if(Code_mere in (select patient_code from patient),'yes','no') as Liaison_mere,left(patient_code,8) as Site_or_lab_code, hospital,Office, if(Network is not null,Network,'Autres') as Network, date_of_birth, gender, start_date as arv_start,if(Office='CAP', 'HUJ',lab) as Lab, date_blood_taken, date_caris_received_sample, date_lab_received_sample, Result,
if(timestampdiff(day,date_of_birth,date_blood_taken)<=61,if(timestampdiff(day,date_of_birth,date_blood_taken)>=0,'0_2',''),if(timestampdiff(day,date_of_birth,date_blood_taken)<=365,if(timestampdiff(day,date_of_birth,date_blood_taken)>61,'2_12',''),'')) as tranche_age
,if(arv is not null,'yes','') as on_arv, timestampdiff(day,date_lab_received_sample,now()) as total_jours_Labo, timestampdiff(day,date_blood_taken,pcr_result_date) as from_specimen_to_result, timestampdiff(day,date_blood_taken,date_caris_received_sample) as from_specimen_to_caris,
timestampdiff(day,date_caris_received_sample,date_lab_received_sample) as from_caris_to_lab, timestampdiff(day,date_lab_received_sample,pcr_result_date) as from_lab_to_result,
if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab")) as days_in_lab,
case 
	when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))>15 then 'Ndays_Labo_more_than_15'
    when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))<=15 then 'Ndays_Labo_equal_or_less_than_15'
    else'not in lab' 
end as Nbres_jours_Labo, 
if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) as days_between_caris_lab,
case 
	when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00') ,timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) >5 then 'Ndays_Caris_more_than_5'
    when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) <=5 then 'Ndays_Caris_equal_or_less_than_5'
end as Nbres_jours_Caris,
if(pcr_result=1,if(patient_info is not null,if(is_abandoned=0 and is_dead=0,'no','yes'),''),'') as pos_dead_abandon
,death_date
,abandoned_date
,pcr_result_date,
case 
    when month(date_blood_taken) between 10 and 12 then 'Q1'
    when month(date_blood_taken) between 1 and 3 then 'Q2'
    when month(date_blood_taken) between 4 and 6 then 'Q3'
    else 'Q4'
End as Quarter,  -- month(date_blood_taken)  as month_blood_taken,
if( month(date_blood_taken) between 10 and 12 ,year(date_blood_taken)+1,year(date_blood_taken) ) as Year
from(
select patient_code,tme.id_patient as id_in_mere ,concat(tme.mother_city_code,'/',tme.mother_hospital_code,'/',tme.mother_code) as Code_mere, date_of_birth, tracking_infant.id_patient as patient_info, date_blood_taken, pcr_result,arv.id_patient as arv, is_dead, is_abandoned, death_date, abandoned_date, pcr_result_date, date_caris_received_sample, date_lab_received_sample,
lookup_testing_specimen_result.name as _result, if(lookup_testing_specimen_result.name="", 'En cours',lookup_testing_specimen_result.name) as Result,lookup_hospital.name as hospital,lookup_network.name as Network,lookup_hospital.office as Office, lookup_testing_gender.name as gender, tracking_regime.start_date, lookup_lab.name as lab from(
    select * from testing_specimen t1 where timestampdiff(day,date_of_birth,date_blood_taken)<=365 and date_blood_taken between '{EidDate.start_date.value}' and '{EidDate.end_date.value}'
    and date_blood_taken=(select min(date_blood_taken) from testing_specimen t2 where t1.id_patient=t2.id_patient)
)test_1
left join patient pa on pa.id=test_1.id_patient
left join testing_mereenfant tme on pa.id= tme.id_patient
left join
(select * from(
    select id_patient from tracking_followup where on_arv=1
    union
    select id_patient from tracking_regime where category='regime_infant_treatment'
    )o
)arv on arv.id_patient=test_1.id_patient
left join tracking_infant on tracking_infant.id_patient=test_1.id_patient
left join lookup_testing_specimen_result on pcr_result=lookup_testing_specimen_result.id
left join lookup_hospital on concat(lookup_hospital.city_code,'/',lookup_hospital.hospital_code)=left(patient_code,8)
left join tracking_regime on tracking_regime.id_patient=test_1.id_patient
left join lookup_testing_gender on lookup_testing_gender.id=gender
left join lookup_lab on lookup_hospital.id_lab=lookup_lab.id
left join lookup_network on lookup_hospital.network=lookup_network.id
where timestampdiff(day,test_1.date_of_birth,test_1.date_blood_taken)<=365 and test_1.date_blood_taken between '{EidDate.start_date.value}' and '{EidDate.end_date.value}' and linked_to_id_patient=0
group by test_1.id_patient
)final
"""

'USE caris_db;'

In [7]:
load_dotenv("../actual_caris_host.env")
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

True

In [8]:
mhc22 = read_sql_query(text(MHC22),engine.connect(),parse_dates=True)
nomhc22 = read_sql_query(text(NOMHC22),engine.connect(),parse_dates=True)
mh22 = read_sql_query(text(MUSO_HOUSEHOLD_22),engine.connect(),parse_dates=True)
eid = read_sql_query(text(EID),engine.connect(),parse_dates=True)
#nomhc22 = read_sql_query(NOMHC22,engine,parse_dates=True)
#all_hub = read_sql_query(HUBS,engine,parse_dates=True)

In [9]:
engine.dispose()

In [10]:
mhc22.to_excel('./mhc22.xlsx',index=False)
nomhc22.to_excel('./nomhc22.xlsx',index=False)
mh22.to_excel('./mh22.xlsx',index=False)
eid.to_excel('./eid.xlsx',index=False)
#all_hub.to_excel('./all_hub.xlsx',index=False)